# Attention is All You Need - NLP 처리 transformer 모델 기본 코드 구성 (Transformer 모듈로 코드 수정)

참고
> https://medium.com/data-science/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

를 바탕으로 GPT와 구현함.

### Import libraries

In [ ]:
# 1. import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math

import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Preparing sample data (sentences)

In [ ]:
# 임의의 자연어 문장 데이터
raw_sentences = [
    "The cat sat on the mat",
    "The dog barked at the cat",
    "The sun is shining brightly",
    "The quick brown fox jumps over the lazy dog",
    "Artificial intelligence is the future of technology",
    "Deep learning models are powerful tools for data analysis",
    "Natural language processing enables machines to understand human language",
    "Transformers have revolutionized the field of machine translation",
    "Recurrent neural networks are useful for sequential data",
    "The weather today is sunny with a chance of rain",
]

# 각 문장을 (입력, 출력) 쌍으로 구성
# 입력은 문장의 단어들, 출력은 다음 단어
# 예: "The cat sat on the" -> "cat sat on the mat"
src_sentences = []
tgt_sentences = []

for sentence in raw_sentences:
    words = sentence.strip().split()
    if len(words) >= 3:
        src_sentences.append(words[:-1])
        tgt_sentences.append(words[1:])

In [ ]:
print(src_sentences[0])
print(tgt_sentences[0])

['The', 'cat', 'sat', 'on', 'the']
['cat', 'sat', 'on', 'the', 'mat']


### Tokenize & Vocabulary

In [ ]:
from collections import Counter

# 1. 문장 -> 단어 단위 토큰화
def tokenize(sentences):
    return sentence.lower().replace('.', '').split()

# 2. (src, tgt) 문장 분리
src_sentences = []
tgt_sentences = []

for sentence in raw_sentences:
    tokens = tokenize(sentence)
    if len(tokens) >= 3:
        src_sentences.append(tokens[:-1]) # 입력: 끝 단어 제외
        tgt_sentences.append(tokens[1:]) # 출력: 시작 단어 제외

# 3. 전체 단어 수집 (단어 사전 생성)
all_tokens = [tokens for sent in src_sentences + tgt_sentences for tokens in sent]
token_freq = Counter(all_tokens)
vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}  # 특수 토큰 포함

for token in token_freq:
    if token not in vocab:
        vocab[token] = len(vocab)

# 4. 인덱스 -> 단어 매핑도 저장
inv_vocab = {idx: token for token, idx in vocab.items()}

# 5. 문장 시퀀스를 정수 인덱스로 변환 (패딩 포함)
def encode(tokens, vocab, max_len):
    ids = [vocab.get(token, vocab['<unk>']) for token in tokens]
    ids = [vocab['<sos>']] + ids + [vocab['<eos>']]  # 시작/종료 토큰 추가
    if len(ids) < max_len:
        ids += [vocab['<pad>']] * (max_len - len(ids))  # 패딩
    else:
        ids = ids[:max_len]
    return ids

# 6. 시퀀스 최대 길이 설정 (특수 토큰 포함)
max_seq_length = 10

src_encoded = [encode(sent, vocab, max_seq_length) for sent in src_sentences]
tgt_encoded = [encode(sent, vocab, max_seq_length) for sent in tgt_sentences]

# 7. 텐서 변환
src_tensor = torch.tensor(src_encoded)
tgt_tensor = torch.tensor(tgt_encoded)

In [ ]:
src_tensor.shape  # torch.Size([10, 10])  ← 10문장, 각 10단어 (패딩 포함)
tgt_tensor.shape  # torch.Size([10, 10])

# vocab = {
#     '<pad>': 0,
#     '<sos>': 1,
#     '<eos>': 2,
#     '<unk>': 3,
#     'the': 4,
#     'cat': 5,
#     ...
# }

torch.Size([10, 10])

### MultiHeadAttention & Position-wise Feed-Forward Networks

PyTorch의 `nn.Transformer` 또는 `nn.TransformerEncoderLayer`/`nn.TransformerDecoderLayer`를 사용할 경우: **MultiHeadAttention, Position-wise Feed-Forward Networks** 구현 필요 x   
1. `nn.Transformer`는 이미 내부에 모든 구성요소를 포함함
  - `MultiHeadAttention` = `nn.MultiheadAttention`
  - `PositionWiseFeedForward` = `nn.Linear -> ReLU -> nn.Linear`
  - `Residual` + `LayerNorm` = 이미 각 sublayer 안에 포함

2. `nn.TransformerEncoderLayer`, `nn.TransformerDecoderLayer`는 **Attention + FFN + LayerNorm** 구조를 기본으로 가짐
  - 사용자가 직접 분리 구현할 필요없이 클래스를 불러서 layer stacking만 하면 됨

3. 직접 구현은 학습 목적 또는 커스터마이징할 경우에만 필요함
  - 예) Attention 타입을 바꾸거나, 구조를 변경하려는 경우

### Positional Encoding

- 기존 코드와 거의 동일
- 차이점: `dropout` 추가
  - Transformer 모델에서 embedding + positional encoding 결과에 `dropout`을 적용하는 이유
    1. 과적합 방지
      - embedding + positional encoding은 모델의 첫 입력이자 모든 레이어를 거쳐 전달되는 정도의 출발점
      - 이 첫 입력을 통해 overfitting 발생 가능성
      - `dropout`으로 입력의 일부를 무작위로 제거해 일반화 성능 향상
    2. 다른 레이어들과의 일관성
      - Transformer의 모든 sub-layer ouput(`attention`, `feed-forward` 등)에는 `dropout` 적용됨
      - positional encoding에도 `dropout`을 걸어주는 건 그 구조와 학습 방식의 일관성 유지에 도움
- transformer에서 dropout 실행해줄 것으로 다시 제거함

In [ ]:
class PositionalEncoding(nn.Module):
    # d_model: 임베딩 차원, max_deq_length: 최대 시퀀스 길이
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        # 전체 시퀀스 길이와 차원 수에 맞는 위치 벡터를 0으로 초기화
        pe = torch.zeros(max_seq_length, d_model)

        # position[0], [1], [2], ..., [max_seq_length-1] 형태로 위치 벡터 생성
        # .unsqueeze(1)은 2D로 변환 (모양: [max_seq_length, 1])
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)

        # sin, cos의 주기를 다르게 해주는 인자
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        # 짝수 인덱스는 sin, 홀수 인덱스는 cos을 사용
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # pe를 학습 파라미터는 아니지만 모델의 일부로 등록
        # 학습 중에는 업데이트 되지 않지만 GPU 메모리에 저장되어 forward 시 사용됨
        self.register_buffer('pe', pe.unsqueeze(0))  # shape: (1, max_seq_len, d_model)

    def forward(self, x):
        # 입력 임베딩 x에 위치 인코딩을 더함
        # self.pe[:, :x.size(1)]는 입력 길이에 맞는 위치 벡터만 사용
        return x + self.pe[:, :x.size(1)] # x shape: (batch_size, seq_len, d_model)

### EncoderLayer & DecoderLayer

- `nn.TransformerEncoderLayer`, `nn.TransformerDecoderLayer`로 구현 가능
- EncoderLayer 포함 유소  
  - MultiHeadAttention, LayerNorm, Dropout, PositionWiseFeedForward
- DecoderLayer 포함 요소
  - Masked MultiHeadAttention, Cross-Attention, LayerNorm, Dropout, PositionWiseFeedForward


### TransformerModel

- PositionalEncoding의 dropout 문제
  - PositionalEncoding class: 위치 정보만 더해주는 역할
  - TransformerModel forward: dropout 추가해줌
    - `src = self.dropout(self.positional_encoding(self.encoder_embedding(src)))`
    - tgt = `self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))`
    


#### generate_mask() 함수 삭제함
- 직접 구현한 transformer 구조의 마스크 형태와 다른 형식 필요
- PyTorch의 `nn.Transformer` 계열 모듈: 세 종류의 마스크를 분리해서 받음
- 따라서 역할별 마스크를 따로 제공   
| 마스크 이름                 | Shape              | 역할                                                             |
|----------------------------|--------------------|------------------------------------------------------------------|
| `tgt_mask`                | (tgt_len, tgt_len) | 미래 차단 (no-peak mask), 디코더의 자기 회귀 형태 유지              |
| `src_key_padding_mask`    | (batch, src_len)   | 소스 문장의 패딩 토큰 무시                                         |
| `tgt_key_padding_mask`    | (batch, tgt_len)   | 타겟 문장의 패딩 토큰 무시                                         |
| `memory_key_padding_mask` | (batch, src_len)   | 인코더 출력 마스킹 (일반적으로 `src_key_padding_mask`와 동일하게 사용) |


In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(TransformerModel, self).__init__()

        # 소스, 타겟 임베딩
        # 1. 소스 문장을 임베딩 벡터로 전환 (단어 ID -> d_model 차원 벡터)
        self.src_embedding = nn.Embedding(src_vocab_size, d_model) # == encoder_embedding
        # 2. 타켓 문장을 임베딩 벡터로 변환
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model) # == decoder_embedding
        # 3. 위치 정보를 임베딩에 추가
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        # 인코더 레이어와 전체 인코더
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            activation='relu', # PositionWiseFeedForward 내부 활성화 함수 지정
            batch_first=True # 입력 텐서의 shape 순서를 (batch, seq_len, d_model)로 고정
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 디코더 레이어와 전체 디코더
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            activation='relu', # PositionWiseFeedForward 내부 활성화 함수 지정
            batch_first=True # 입력 텐서의 shape 순서를 (batch, seq_len, d_model)로 고정
        )
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        # 최종 출력 → 단어 분포
        self.fc_out = nn.Linear(d_model, tgt_vocab_size) # linear

        # 드롭아웃
        self.dropout = nn.Dropout(dropout)


    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None, tgt_mask=None):
        # 1. 임베딩 + 위치 인코딩
        src_emb = self.dropout(self.positional_encoding(self.src_embedding(src)))  # (batch, seq, d_model)
        tgt_emb = self.dropout(self.positional_encoding(self.tgt_embedding(tgt)))

        # 2. 인코더
        memory = self.encoder(src_emb, src_key_padding_mask=src_key_padding_mask)

        # 3. 디코더
        output = self.decoder(tgt_emb, memory,
                              tgt_mask=tgt_mask,
                              tgt_key_padding_mask=tgt_key_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask)

        # 4. 최종 출력층
        return self.fc_out(output)  # (batch, tgt_seq_len, vocab_size)

### Hyperparameter Setting and Model Initialization

In [ ]:
# 1. 하이퍼파라미터 설정 (vocab과 시퀀스 길이에 맞춤)
src_vocab_size = len(vocab)       # 단어 사전 크기
tgt_vocab_size = len(vocab)       # 동일하게 사용
d_model = 512                     # 임베딩 차원
num_heads = 8                     # Multi-head 수
num_layers = 6                    # 인코더/디코더 레이어 수
d_ff = 2048                       # FeedForward 차원
max_seq_length = 10              # 우리가 encode할 때 사용한 길이
dropout = 0.1                     # 드롭아웃 비율

# 2. Transformer 모델 초기화
model = TransformerModel(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout
)

# 3. 영어 문장에서 생성된 실제 입력 데이터 사용
src_data = src_tensor  # shape: (10, 10)
tgt_data = tgt_tensor  # shape: (10, 10)
batch_size = src_data.size(0)  # = 10

### Training the model

In [9]:
# 손실 함수 정의
criterion = nn.CrossEntropyLoss(ignore_index=vocab['<pad>'])

# 옵티마이저 정의
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# 모델 학습 모드로 전환
model.train()

# No-Peak 마스크 생성 함수 (디코더용 future masking)
def generate_square_subsequent_mask(seq_len):
    return torch.triu(torch.full((seq_len, seq_len), float('-inf')), diagonal=1)

# 총 10 에포크 동안 학습
for epoch in range(10):
    optimizer.zero_grad()

    # 1. 입력 마스크 생성
    seq_len = tgt_data.size(1)
    tgt_mask = generate_square_subsequent_mask(seq_len - 1).to(tgt_data.device)

    src_key_padding_mask = (src_data == vocab['<pad>'])          # (batch, src_len)
    tgt_key_padding_mask = (tgt_data[:, :-1] == vocab['<pad>'])  # (batch, tgt_len - 1)

    # 2. 디코더 입력/정답 분리
    decoder_input = tgt_data[:, :-1]     # <sos> A B
    target_output = tgt_data[:, 1:]      # A B <eos>

    # 3. 모델 실행
    output = model(
        src=src_data,
        tgt=decoder_input,
        src_key_padding_mask=src_key_padding_mask,
        tgt_key_padding_mask=tgt_key_padding_mask,
        memory_key_padding_mask=src_key_padding_mask,
        tgt_mask=tgt_mask
    )

    # 4. 손실 계산 (출력: (batch, seq, vocab) → reshape)
    loss = criterion(output.view(-1, len(vocab)), target_output.reshape(-1))

    # 5. 역전파 및 업데이트
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch 1, Loss: 4.2341
Epoch 2, Loss: 3.9380
Epoch 3, Loss: 3.9025
Epoch 4, Loss: 3.7953
Epoch 5, Loss: 3.6977
Epoch 6, Loss: 3.5489
Epoch 7, Loss: 3.4313
Epoch 8, Loss: 3.2201
Epoch 9, Loss: 2.8779
Epoch 10, Loss: 2.7013


### Transformer prediction -> Post-processing

In [12]:
# 1. softmax 확률 분포 계산
probs = torch.softmax(output, dim=-1)  # 확률 분포

# 2. 예측 인덱스 선택: argmax
predicted_ids = output.argmax(dim=-1)  # shape: (batch, seq_len)

# 3. 디코딩 함수 정의
def decode_indices(indices, inv_vocab):
    tokens = []
    for idx in indices:
        idx = idx.item()
        if idx == vocab['<eos>']:
            break
        if idx != vocab['<pad>'] and idx != vocab['<sos>']:
            tokens.append(inv_vocab.get(idx, '<unk>'))
    return " ".join(tokens)

# 5. 배치 전체 예측 디코딩
decoded_sentences = []
for i in range(predicted_ids.size(0)):
    decoded = decode_indices(predicted_ids[i], inv_vocab)
    decoded_sentences.append(decoded)

# 6. 예측 결과 비교 출력
print("\n[예측 결과]")
for i in range(len(decoded_sentences)):
    input_text = decode_indices(src_data[i], inv_vocab)
    target_text = decode_indices(tgt_data[i][1:], inv_vocab)  # 정답 시퀀스
    print(f"[입력   {i+1}] {input_text}")
    print(f"[정답   {i+1}] {target_text}")
    print(f"[예측   {i+1}] {decoded_sentences[i]}")
    print()


[예측 결과]
[입력   1] the cat sat on the
[정답   1] cat sat on the mat
[예측   1] the the the the the the the the cat

[입력   2] the dog barked at the
[정답   2] dog barked at the cat
[예측   2] the the the the the

[입력   3] the sun is shining
[정답   3] sun is shining brightly
[예측   3] is is is is is is is is is

[입력   4] the quick brown fox jumps over the lazy
[정답   4] quick brown fox jumps over the lazy dog
[예측   4] the dog dog the the the lazy dog

[입력   5] artificial intelligence is the future of
[정답   5] intelligence is the future of technology
[예측   5] technology technology future technology technology technology technology technology future

[입력   6] deep learning models are powerful tools for data
[정답   6] learning models are powerful tools for data analysis
[예측   6] are are are are for for for for data

[입력   7] natural language processing enables machines to understand human
[정답   7] language processing enables machines to understand human language
[예측   7] language language language langu

In [ ]:
# # 7. prediction -> post processing (정석 적용)
# model.eval()  # 평가 모드로 전환

# with torch.no_grad():  # 그래디언트 계산 생략 (속도 + 메모리 절약)
#     output = model(
#         src=src_data,
#         tgt=tgt_data[:, :-1],
#         src_key_padding_mask=(src_data == vocab['<pad>']),
#         tgt_key_padding_mask=(tgt_data[:, :-1] == vocab['<pad>']),
#         memory_key_padding_mask=(src_data == vocab['<pad>']),
#         tgt_mask=generate_square_subsequent_mask(tgt_data.size(1) - 1).to(device)
#     )

# # 1. softmax 확률 분포 계산
# probs = torch.softmax(output, dim=-1)  # 확률 분포

# # 2. 예측 인덱스 선택: argmax
# predicted_ids = output.argmax(dim=-1)  # shape: (batch, seq_len)

# # 3. 디코딩 함수 정의
# def decode_indices(indices, inv_vocab):
#     tokens = []
#     for idx in indices:
#         idx = idx.item()
#         if idx == vocab['<eos>']:
#             break
#         if idx != vocab['<pad>'] and idx != vocab['<sos>']:
#             tokens.append(inv_vocab.get(idx, '<unk>'))
#     return " ".join(tokens)

# # 5. 배치 전체 예측 디코딩
# decoded_sentences = []
# for i in range(predicted_ids.size(0)):
#     decoded = decode_indices(predicted_ids[i], inv_vocab)
#     decoded_sentences.append(decoded)

# # 6. 예측 결과 비교 출력
# print("\n[예측 결과]")
# for i in range(len(decoded_sentences)):
#     input_text = decode_indices(src_data[i], inv_vocab)
#     target_text = decode_indices(tgt_data[i][1:], inv_vocab)  # 정답 시퀀스
#     print(f"[입력   {i+1}] {input_text}")
#     print(f"[정답   {i+1}] {target_text}")
#     print(f"[예측   {i+1}] {decoded_sentences[i]}")
#     print()


### END